# 反事实多选项生成系统
## Counterfactual Multiple-Choice Options Generator for Accident Videos

用于从事故视频和QA对生成高质量的反事实选项（错误选项），构建标准化的多选题。

**关键特性:**
- 调用 Qwen VL API 生成反事实选项
- 自动计算与正确答案的相似度
- 支持可变选项数（3, 4, 5）用于消融实验
- 输出相似度统计和质量评分

In [ ]:
# ============================================================================
# 部分 1: 安装依赖和导入
# ============================================================================

# 安装必需的包
%pip install -q pandas numpy scikit-learn matplotlib seaborn tqdm openai torch torchvision
%pip install -q nltk python-dotenv

In [ ]:
# 导入库
import os
import json
import re
import pandas as pd
import numpy as np
import cv2
from pathlib import Path
from typing import Dict, List, Optional, Tuple
from dataclasses import dataclass, asdict
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# 相似度计算
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# 进度条
from tqdm import tqdm

# 绘图
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")

print("✅ 所有依赖导入成功")

In [ ]:
# ============================================================================
# 部分 2: 配置和常量
# ============================================================================

@dataclass
class Config:
    """全局配置"""
    # API 配置
    qwen_api_key: str = None
    qwen_api_url: str = "https://dashscope-intl.aliyuncs.com/compatible-mode/v1"
    
    # 文件路径
    qa_csv_path: str = "/home/24068286g/UString/VRU/src/option_generate/data/QA_pair.csv"
    video_dir: str = "/home/24068286g/UString/data/crash/videos/Crash-1500"
    output_dir: str = "/home/24068286g/UString/VRU/src/option_generate/outputs"
    
    # 处理配置
    num_options_list: List[int] = None  # [3, 4, 5] 用于消融实验
    max_frames_per_video: int = 8
    target_fps: int = 2
    
    # 相似度配置
    min_similarity: float = 0.2  # 选项不应过于相似
    max_similarity: float = 0.8  # 选项不应过于相似
    similarity_threshold: float = 0.5  # 用于判断是否为有效选项
    
    # API 参数
    api_timeout: int = 60
    max_retries: int = 3
    batch_size: int = 1
    
    def __post_init__(self):
        # 从 Colab Secrets 加载 API Key
        try:
            from google.colab import userdata
            if self.qwen_api_key is None:
                self.qwen_api_key = userdata.get('ALI_INTERNATIONAL_KEY')
        except:
            pass
        
        # 从环境变量加载
        if self.qwen_api_key is None:
            self.qwen_api_key = os.getenv('ALI_INTERNATIONAL_KEY')
        
        # 初始化默认选项数
        if self.num_options_list is None:
            self.num_options_list = [3, 4, 5]
        
        # 创建输出目录
        Path(self.output_dir).mkdir(parents=True, exist_ok=True)

# 创建全局配置实例
config = Config()

# 验证配置
print("🔧 系统配置")
print("=" * 70)
print(f"✓ QA CSV: {config.qa_csv_path}")
print(f"✓ 视频目录: {config.video_dir}")
print(f"✓ 输出目录: {config.output_dir}")
print(f"✓ API Key: {'✅ 已加载' if config.qwen_api_key else '❌ 未加载'}")
print(f"✓ 选项数量: {config.num_options_list}")
print(f"✓ 相似度范围: [{config.min_similarity}, {config.max_similarity}]")
print("=" * 70)

In [ ]:
# ============================================================================
# 部分 3: 加载 QA 数据
# ============================================================================

def load_qa_data(csv_path: str) -> pd.DataFrame:
    """
    加载 QA_pair.csv 文件
    """
    print(f"📂 加载 QA 数据: {csv_path}")
    
    try:
        df = pd.read_csv(csv_path)
        print(f"  ✓ 成功加载 {len(df)} 个视频的 QA 对")
        print(f"  ✓ 列数: {len(df.columns)}")
        print(f"\n  字段示例: {df.columns[:10].tolist()}")
        return df
    except Exception as e:
        print(f"  ❌ 加载失败: {e}")
        return None

def extract_qa_pairs(df: pd.DataFrame) -> List[Dict]:
    """
    从 DataFrame 中提取 QA 对
    返回格式: [
        {
            "video_id": "000003",
            "questions": [
                {
                    "q_id": 1,
                    "text": "问题文本",
                    "category": "分类",
                    "correct_answer": "正确答案",
                    "wrong_options": ["错选1", "错选2", "错选3"]
                },
                ...
            ]
        },
        ...
    ]
    """
    qa_data = []
    
    for idx, row in df.iterrows():
        video_id = str(row.get('video_number', idx)).zfill(6)
        
        # 提取问题 (q1, q2, q3, q4, q5, q6)
        questions = []
        for q_num in range(1, 7):
            q_col = f'q{q_num}_text'
            ans_col = f'q{q_num}_ans_correct'
            cat_col = f'q{q_num}_category'
            
            if q_col in df.columns and pd.notna(row.get(q_col)):
                question = {
                    'q_id': q_num,
                    'text': str(row[q_col]).strip(),
                    'category': str(row.get(cat_col, 'unknown')).strip(),
                    'correct_answer': str(row.get(ans_col, '')).strip(),
                    'wrong_options': []
                }
                
                # 收集错误选项
                for w_num in range(1, 4):
                    wrong_col = f'q{q_num}_ans_wrong{w_num}'
                    if wrong_col in df.columns and pd.notna(row.get(wrong_col)):
                        wrong_ans = str(row[wrong_col]).strip()
                        if wrong_ans and wrong_ans.lower() != 'nan':
                            question['wrong_options'].append(wrong_ans)
                
                if question['text'] and question['correct_answer']:
                    questions.append(question)
        
        if questions:
            qa_data.append({
                'video_id': video_id,
                'questions': questions
            })
    
    return qa_data

# 加载数据
print("\n📋 加载 QA 数据集")
print("=" * 70)
qa_df = load_qa_data(config.qa_csv_path)

if qa_df is not None:
    qa_pairs = extract_qa_pairs(qa_df)
    print(f"\n✅ 成功提取 {len(qa_pairs)} 个视频的 QA 对")
    print(f"   总问题数: {sum(len(v['questions']) for v in qa_pairs)}")
    
    # 显示样例
    if qa_pairs:
        print(f"\n📌 示例 (视频 {qa_pairs[0]['video_id']})")
        for q in qa_pairs[0]['questions'][:1]:
            print(f"  问: {q['text'][:60]}...")
            print(f"  答: {q['correct_answer'][:60]}...")
            print(f"  分类: {q['category']}")
else:
    qa_pairs = []

In [ ]:
# ============================================================================
# 部分 4: 视频帧加载
# ============================================================================

def load_video_frames(video_path: str, max_frames: int = 8, target_fps: int = 2) -> Optional[np.ndarray]:
    """
    加载视频帧 (均匀采样)
    返回: (num_frames, H, W, 3) 的 numpy 数组或 None
    """
    try:
        cap = cv2.VideoCapture(video_path)
        
        if not cap.isOpened():
            print(f"  ❌ 无法打开: {Path(video_path).name}")
            return None
        
        fps = cap.get(cv2.CAP_PROP_FPS)
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        
        if fps <= 0 or total_frames <= 0:
            cap.release()
            return None
        
        # 计算采样间隔
        frame_interval = max(1, int(fps / target_fps))
        frames = []
        frame_count = 0
        
        while cap.isOpened() and len(frames) < max_frames:
            ret, frame = cap.read()
            if not ret:
                break
            
            if frame_count % frame_interval == 0:
                try:
                    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                    frames.append(frame_rgb)
                except:
                    pass
            
            frame_count += 1
        
        cap.release()
        
        if frames:
            return np.array(frames, dtype=np.uint8)
        return None
        
    except Exception as e:
        print(f"  ❌ 加载视频出错: {e}")
        return None

def find_video_file(video_id: str, video_dir: str) -> Optional[str]:
    """
    根据视频 ID 查找视频文件
    """
    video_dir = Path(video_dir)
    
    # 尝试多种扩展名
    for ext in ['.mp4', '.avi', '.mov', '.mkv']:
        video_path = video_dir / f"{video_id}{ext}"
        if video_path.exists():
            return str(video_path)
    
    return None

print("✅ 视频加载函数定义完成")

In [ ]:
# ============================================================================
# 部分 5: 提示词模板
# ============================================================================

def create_option_generation_prompt(
    question: str,
    correct_answer: str,
    num_options: int,
    video_context: str = "",
    question_category: str = ""
) -> str:
    """
    创建用于生成反事实选项的提示词
    
    关键策略:
    1. 要求选项与正确答案语义相关但不同
    2. 要求选项与问题上下文一致
    3. 要求选项具有合理的难度
    4. 避免生成过于明显的错误选项
    """
    
    prompt = f"""你是一个多选题出题专家，专门为视频理解任务生成高质量的错误选项（干扰项）。

【任务】为以下问题生成 {num_options - 1} 个高质量的错误选项。

【问题】
{question}

【正确答案】
{correct_answer}

【视频背景】（如果有）
{video_context if video_context else "事故驾驶视频"}

【问题分类】
{question_category if question_category else "通用"}

【要求】

1. **相似度平衡**：
   - 错误选项应该与正确答案在语义上相关（涉及相同的主题或概念）
   - 但表述和内容应该有明显区别（不能过于接近）
   - 例如：如果正确答案涉及"T型碰撞"，选项可以涉及"侧滑碰撞"、"追尾碰撞"等

2. **与问题的一致性**：
   - 所有选项（包括错误选项）都应该直接回答问题
   - 选项应该符合问题的语境和上下文
   - 避免生成无关的或语法错误的选项

3. **合理的难度**：
   - 选项应该对学生有一定的迷惑性，但不能是荒谬的
   - 应该反映常见的错误理解或混淆点
   - 避免过于相似或过于差异

4. **格式要求**：
   - 使用清晰、简洁的语言
   - 长度与正确答案相近（±20%）
   - 避免使用极端词汇或过度修饰

【生成指南】

根据以下策略生成 {num_options - 1} 个错误选项：

"""
    
    if num_options == 3:
        prompt += """选项1：主要错误角度（与正确答案涉及不同的场景元素）
选项2：常见混淆点（学生容易混淆的相关概念）
"""
    elif num_options == 4:
        prompt += """选项1：相似但不同的场景（涉及类似类型但细节不同的碰撞/事件）
选项2：常见混淆点（学生容易混淆的概念）
选项3：局部正确但整体错误（答案中某些部分正确，但整体理解有误）
"""
    elif num_options == 5:
        prompt += """选项1：相似类型的碰撞/事件（不同于正确答案但相关）
选项2：颠倒因果关系（混淆原因和结果）
选项3：部分正确的答案（混合正确和错误信息）
选项4：常见学生错误（基于实际教学经验的错误理解）
"""
    
    prompt += f"""

【输出格式】

只输出 {num_options - 1} 个错误选项，每个选项单独一行，不需要编号或额外标记。
例如：
第一个选项内容
第二个选项内容
第三个选项内容（如果适用）

每个选项应该是一个完整的句子或短段落，能够作为多选题的独立选项。

【最后提醒】

- 确保生成的选项与正确答案有语义关联（至少50%相似度）
- 但又有足够的区别（不超过80%相似度）
- 所有选项应该看起来都是合理的答案，但只有正确答案是真正正确的
"""
    
    return prompt

# 测试提示词
print("📝 提示词生成函数已定义")
print("\n📌 样例提示词 (num_options=4):")
print("=" * 70)
sample_prompt = create_option_generation_prompt(
    question="What type of collision occurs between the two vehicles?",
    correct_answer="A T-bone (side-impact) collision where one vehicle hits the side of another",
    num_options=4,
    video_context="Urban traffic intersection with traffic lights",
    question_category="Accident Type"
)
print(sample_prompt[:500] + "\n...(省略)...")

In [ ]:
# ============================================================================
# 部分 6: Qwen VL API 调用
# ============================================================================

import base64
from io import BytesIO
from PIL import Image
import time
from openai import OpenAI
import re

class QwenVLOptionGenerator:
    """使用 Qwen VL API 生成反事实选项"""
    
    def __init__(self, api_key: str, api_url: str = "https://dashscope-intl.aliyuncs.com/compatible-mode/v1"):
        self.api_key = api_key
        self.api_url = api_url
        self.client = OpenAI(api_key=api_key, base_url=api_url)
        self.retry_count = 0
    
    def frames_to_base64(self, frames: np.ndarray, max_frames: int = 4) -> List[str]:
        """
        将视频帧转换为 base64 编码（用于 API 调用）
        """
        b64_list = []
        
        # 均匀采样帧
        if len(frames) > max_frames:
            indices = np.linspace(0, len(frames) - 1, max_frames, dtype=int)
            selected_frames = frames[indices]
        else:
            selected_frames = frames
        
        for frame in selected_frames:
            try:
                img = Image.fromarray(frame)
                buf = BytesIO()
                img.save(buf, format="JPEG", quality=85)
                b64 = base64.b64encode(buf.getvalue()).decode()
                b64_list.append(b64)
            except Exception as e:
                print(f"    ⚠️ 帧转换失败: {e}")
                continue
        
        return b64_list
    
    def generate_options(
        self,
        question: str,
        correct_answer: str,
        num_options: int,
        video_frames: Optional[np.ndarray] = None,
        video_context: str = "",
        question_category: str = ""
    ) -> Tuple[List[str], float, Optional[str]]:
        """
        调用 Qwen VL API 生成反事实选项
        
        返回: (选项列表, 处理时间, 错误信息)
        """
        
        try:
            # 创建提示词
            prompt_text = create_option_generation_prompt(
                question=question,
                correct_answer=correct_answer,
                num_options=num_options,
                video_context=video_context,
                question_category=question_category
            )
            
            start_time = time.time()
            
            # 构建消息内容
            content = [{"type": "text", "text": prompt_text}]
            
            # 如果有视频帧，添加图像
            if video_frames is not None and len(video_frames) > 0:
                b64_frames = self.frames_to_base64(video_frames, max_frames=4)
                for b64 in b64_frames:
                    content.append({
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{b64}"
                        }
                    })
            
            # 调用 API
            response = self.client.chat.completions.create(
                model="qwen-vl-max",
                messages=[{"role": "user", "content": content}],
                temperature=0.7,
                top_p=0.9,
                max_tokens=1000,
                timeout=config.api_timeout
            )
            
            elapsed_time = time.time() - start_time
            
            # 提取响应
            response_text = response.choices[0].message.content
            
            # 解析选项
            options = self._parse_options(response_text, num_options - 1)
            
            return options, elapsed_time, None
            
        except Exception as e:
            error_msg = str(e)
            elapsed_time = time.time() - start_time if 'start_time' in locals() else 0
            print(f"    ❌ API 错误: {error_msg[:100]}")
            return [], elapsed_time, error_msg
    
    def _parse_options(self, response_text: str, expected_count: int) -> List[str]:
        """
        从 API 响应中解析选项（改进版 - 避免截断开头）
        """
        lines = response_text.strip().split('\n')
        
        # 过滤空行
        options = [line.strip() for line in lines if line.strip()]
        
        # 移除可能的编号前缀（更精确的方式）
        cleaned_options = []
        for opt in options:
            # 使用正则表达式只移除明确的编号模式
            # 匹配: "1.", "1)", "A.", "A)", "(1)", "[1]" 等
            opt_cleaned = re.sub(r'^[\[\(]?\d+[\]\)]?[\.\)\:\-\s]+|^[A-Za-z][\.\)\:\-\s]+', '', opt)
            
            # 如果清理后为空或过短，使用原始文本
            if len(opt_cleaned.strip()) < 10:
                opt_cleaned = opt
            
            # 过滤过短的文本（但不要太严格）
            if len(opt_cleaned.strip()) > 5:
                cleaned_options.append(opt_cleaned.strip())
        
        return cleaned_options[:expected_count]

# 初始化生成器
if config.qwen_api_key:
    option_generator = QwenVLOptionGenerator(config.qwen_api_key, config.qwen_api_url)
    print("✅ Qwen VL 选项生成器已初始化")
else:
    option_generator = None
    print("❌ 未找到 API Key，无法初始化生成器")

In [ ]:
# ============================================================================
# 部分 7: 相似度计算
# ============================================================================

class SimilarityCalculator:
    """计算选项与正确答案之间的相似度"""
    
    def __init__(self):
        self.vectorizer = TfidfVectorizer(
            max_features=500,
            lowercase=True,
            stop_words='english'
        )
    
    def compute_similarity(self, correct_answer: str, options: List[str]) -> Dict[str, float]:
        """
        计算每个选项与正确答案的余弦相似度
        
        返回: {"option_text": similarity_score, ...}
        """
        
        if not options:
            return {}
        
        # 创建文本列表
        texts = [correct_answer] + options
        
        try:
            # TF-IDF 向量化
            tfidf_matrix = self.vectorizer.fit_transform(texts)
            
            # 计算第一个（正确答案）与其他的相似度
            similarities = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:])[0]
            
            # 创建结果字典
            result = {}
            for i, opt in enumerate(options):
                result[opt] = float(similarities[i])
            
            return result
            
        except Exception as e:
            print(f"    ⚠️ 相似度计算失败: {e}")
            return {}
    
    def evaluate_option_quality(
        self,
        correct_answer: str,
        generated_options: List[str],
        min_sim: float = 0.2,
        max_sim: float = 0.8
    ) -> Tuple[List[str], Dict[str, float]]:
        """
        根据相似度阈值评估和过滤选项
        
        返回: (有效选项, 相似度字典)
        """
        
        similarities = self.compute_similarity(correct_answer, generated_options)
        
        # 过滤选项
        valid_options = []
        for opt, sim in similarities.items():
            if min_sim <= sim <= max_sim:
                valid_options.append(opt)
        
        return valid_options, similarities

# 初始化相似度计算器
similarity_calculator = SimilarityCalculator()
print("✅ 相似度计算器已初始化")

In [ ]:
# ============================================================================
# 部分 8: 主处理管道
# ============================================================================

def process_single_question(
    video_id: str,
    question_data: Dict,
    video_frames: Optional[np.ndarray],
    num_options_list: List[int],
    dense_caption: str = ""
) -> Dict:
    """
    处理单个问题，为其生成反事实选项
    
    返回包含所有选项数量配置的结果
    """
    
    result = {
        'video_id': video_id,
        'q_id': question_data['q_id'],
        'question': question_data['text'],
        'category': question_data['category'],
        'correct_answer': question_data['correct_answer'],
        'timestamp': datetime.now().isoformat(),
        'results_by_num_options': {}
    }
    
    # 为每个选项数量配置生成选项
    for num_options in num_options_list:
        num_options_key = f"{num_options}_options"
        
        print(f"      生成 {num_options} 个选项... ", end="", flush=True)
        
        try:
            # 调用 Qwen VL API
            generated_options, elapsed_time, error = option_generator.generate_options(
                question=question_data['text'],
                correct_answer=question_data['correct_answer'],
                num_options=num_options,
                video_frames=video_frames,
                video_context=dense_caption,
                question_category=question_data['category']
            )
            
            if error:
                print(f"❌ (API 错误)")
                result['results_by_num_options'][num_options_key] = {
                    'status': 'failed',
                    'error': error,
                    'elapsed_time': elapsed_time
                }
                continue
            
            # 计算相似度
            valid_options, similarities = similarity_calculator.evaluate_option_quality(
                correct_answer=question_data['correct_answer'],
                generated_options=generated_options,
                min_sim=config.min_similarity,
                max_sim=config.max_similarity
            )
            
            # 如果有效选项不足，使用所有生成的选项
            final_options = valid_options if len(valid_options) >= num_options - 1 else generated_options[:num_options - 1]
            
            # 计算统计信息
            sims_values = [similarities.get(opt, 0.5) for opt in final_options]
            
            result['results_by_num_options'][num_options_key] = {
                'status': 'success',
                'generated_options': final_options,
                'similarities': {opt: similarities.get(opt, 0.5) for opt in final_options},
                'similarity_stats': {
                    'min': float(np.min(sims_values)) if sims_values else 0.0,
                    'max': float(np.max(sims_values)) if sims_values else 0.0,
                    'mean': float(np.mean(sims_values)) if sims_values else 0.0,
                    'std': float(np.std(sims_values)) if len(sims_values) > 1 else 0.0
                },
                'elapsed_time': elapsed_time
            }
            
            print(f"✅ ({len(final_options)} 选项, mean_sim={np.mean(sims_values):.3f})")
            
        except Exception as e:
            print(f"❌ (异常: {str(e)[:50]})")
            result['results_by_num_options'][num_options_key] = {
                'status': 'failed',
                'error': str(e),
                'elapsed_time': 0
            }
    
    return result

def process_video_batch(qa_data_list: List[Dict], num_options_list: List[int]) -> List[Dict]:
    """
    批量处理所有视频的所有问题
    """
    
    all_results = []
    
    for video_data in tqdm(qa_data_list, desc="处理视频"):
        video_id = video_data['video_id']
        questions = video_data['questions']
        
        print(f"\n🎬 视频 {video_id}")
        
        # 加载视频帧
        video_path = find_video_file(video_id, config.video_dir)
        if not video_path:
            print(f"  ❌ 视频文件未找到")
            continue
        
        video_frames = load_video_frames(video_path, config.max_frames_per_video, config.target_fps)
        if video_frames is None:
            print(f"  ❌ 无法加载视频帧")
            continue
        
        print(f"  ✓ 加载 {len(video_frames)} 帧")
        
        # 处理每个问题
        for q_idx, question in enumerate(questions, 1):
            print(f"  问题 {q_idx}/{len(questions)}: {question['text'][:50]}...")
            
            q_result = process_single_question(
                video_id=video_id,
                question_data=question,
                video_frames=video_frames,
                num_options_list=num_options_list
            )
            
            all_results.append(q_result)
    
    return all_results

print("✅ 处理管道函数已定义")

In [ ]:
# ============================================================================
# 部分 9: 运行和保存结果
# ============================================================================

# 快速测试 (仅处理前 2 个视频, 每个 2 个问题)
print("\n🚀 开始处理")
print("=" * 70)

if option_generator is None:
    print("❌ 错误: Qwen VL 生成器未初始化")
    print("   请确保 ALI_INTERNATIONAL_KEY 已在 Colab Secrets 中设置")
else:
    # 限制数据规模用于测试
    test_qa_data = []
    for video_data in qa_pairs[:2]:  # 仅前 2 个视频
        limited_questions = video_data.copy()
        limited_questions['questions'] = video_data['questions'][:2]  # 每个视频仅 2 个问题
        test_qa_data.append(limited_questions)
    
    print(f"📝 计划处理:")
    print(f"   视频数: {len(test_qa_data)}")
    print(f"   总问题数: {sum(len(v['questions']) for v in test_qa_data)}")
    print(f"   选项数配置: {config.num_options_list}")
    print(f"   总 API 调用次数: {sum(len(v['questions']) for v in test_qa_data) * len(config.num_options_list)}")
    print()
    
    # 处理
    results = process_video_batch(test_qa_data, config.num_options_list)
    
    print(f"\n✅ 处理完成")
    print(f"   生成 {len(results)} 个问题的结果")
    
    # 保存结果
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    
    # JSON 格式
    json_output_path = Path(config.output_dir) / f"generated_options_{timestamp}.json"
    with open(json_output_path, 'w', encoding='utf-8') as f:
        json.dump(results, f, indent=2, ensure_ascii=False)
    print(f"   ✓ JSON 结果保存: {json_output_path}")
    
    # CSV 格式（改进版 - 每个选项单独一列）
    csv_data = []
    for result in results:
        for num_opts_key, num_opts_result in result['results_by_num_options'].items():
            if num_opts_result['status'] == 'success':
                num_options = int(num_opts_key.split('_')[0])
                generated_opts = num_opts_result['generated_options']
                
                # 构建行数据，每个选项单独一列
                row_data = {
                    'video_id': result['video_id'],
                    'question_id': result['q_id'],
                    'question': result['question'],
                    'category': result['category'],
                    'correct_answer': result['correct_answer'],
                    'num_options': num_options,
                }
                
                # 为每个选项创建单独的列
                for i in range(num_options - 1):
                    if i < len(generated_opts):
                        row_data[f'option_{i+1}'] = generated_opts[i]
                        row_data[f'option_{i+1}_similarity'] = num_opts_result['similarities'].get(generated_opts[i], 0.0)
                    else:
                        row_data[f'option_{i+1}'] = None
                        row_data[f'option_{i+1}_similarity'] = None
                
                # 添加统计信息
                row_data.update({
                    'min_similarity': num_opts_result['similarity_stats']['min'],
                    'max_similarity': num_opts_result['similarity_stats']['max'],
                    'mean_similarity': num_opts_result['similarity_stats']['mean'],
                    'std_similarity': num_opts_result['similarity_stats']['std'],
                    'elapsed_time': num_opts_result['elapsed_time']
                })
                
                csv_data.append(row_data)
    
    if csv_data:
        csv_output_path = Path(config.output_dir) / f"generated_options_{timestamp}.csv"
        csv_df = pd.DataFrame(csv_data)
        csv_df.to_csv(csv_output_path, index=False, encoding='utf-8', quoting=1, line_terminator='\n')
        print(f"   ✓ CSV 结果保存: {csv_output_path}")
        print(f"     每个选项单独一列，包含相似度分数")
        print(f"\n📋 CSV 列结构:")
        print(f"   基础列: video_id, question_id, question, category, correct_answer, num_options")
        print(f"   选项列: option_1, option_1_similarity, option_2, option_2_similarity, ...")
        print(f"   统计列: min_similarity, max_similarity, mean_similarity, std_similarity, elapsed_time")
    
    print(f"\n📊 输出目录: {config.output_dir}")

In [ ]:
# ============================================================================
# 部分 10: 消融实验分析
# ============================================================================
# import timestamp
results_json_path = "/content/drive/MyDrive/data/VQA_outputs/full_generated_options_20251204_214859.json"

# 可选：从已生成的 JSON 结果恢复（防止断开后变量丢失）
# results_json_path = None
if 'results' not in locals() and results_json_path:
    with open(results_json_path, 'r', encoding='utf-8') as f:
        results = json.load(f)
    print(f"🔁 已从 JSON 加载 results: {results_json_path}")

def analyze_ablation_results(results: List[Dict]) -> Dict:
    """
    分析消融实验结果 (3 vs 4 vs 5 个选项)
    """

    analysis = {
        'by_num_options': {},
        'by_category': {},
        'overall_stats': {}
    }

    # 按选项数统计
    for num_opts in [3, 4, 5]:
        num_opts_key = f"{num_opts}_options"
        stats = {
            'total_questions': 0,
            'successful': 0,
            'failed': 0,
            'mean_similarity': [],
            'mean_elapsed_time': 0,
            'difficulty_distribution': {
                'easy': 0,    # mean_sim > 0.6
                'medium': 0,  # 0.4 < mean_sim <= 0.6
                'hard': 0     # mean_sim <= 0.4
            }
        }

        elapsed_times = []

        for result in results:
            if num_opts_key in result['results_by_num_options']:
                num_opts_result = result['results_by_num_options'][num_opts_key]
                stats['total_questions'] += 1

                if num_opts_result['status'] == 'success':
                    stats['successful'] += 1
                    mean_sim = num_opts_result['similarity_stats']['mean']
                    stats['mean_similarity'].append(mean_sim)

                    # 难度分类
                    if mean_sim > 0.6:
                        stats['difficulty_distribution']['easy'] += 1
                    elif mean_sim > 0.4:
                        stats['difficulty_distribution']['medium'] += 1
                    else:
                        stats['difficulty_distribution']['hard'] += 1

                    elapsed_times.append(num_opts_result['elapsed_time'])
                else:
                    stats['failed'] += 1

        if stats['mean_similarity']:
            stats['mean_similarity_avg'] = float(np.mean(stats['mean_similarity']))
            stats['mean_similarity_std'] = float(np.std(stats['mean_similarity']))

        if elapsed_times:
            stats['mean_elapsed_time'] = float(np.mean(elapsed_times))

        analysis['by_num_options'][num_opts_key] = stats

    # 按分类统计
    categories = set()
    for result in results:
        categories.add(result['category'])

    for category in categories:
        analysis['by_category'][category] = {}
        for num_opts in [3, 4, 5]:
            num_opts_key = f"{num_opts}_options"
            cat_results = [r for r in results if r['category'] == category
                         and num_opts_key in r['results_by_num_options']]

            if cat_results:
                successful = sum(1 for r in cat_results
                               if r['results_by_num_options'][num_opts_key]['status'] == 'success')
                mean_sims = [r['results_by_num_options'][num_opts_key]['similarity_stats']['mean']
                           for r in cat_results
                           if r['results_by_num_options'][num_opts_key]['status'] == 'success']

                analysis['by_category'][category][num_opts_key] = {
                    'total': len(cat_results),
                    'successful': successful,
                    'success_rate': successful / len(cat_results),
                    'mean_similarity': float(np.mean(mean_sims)) if mean_sims else 0.0
                }

    # 总体统计
    all_successful = sum(r['results_by_num_options'][f"{n}_options"]['status'] == 'success'
                        for r in results
                        for n in [3, 4, 5]
                        if f"{n}_options" in r['results_by_num_options'])

    all_total = sum(1 for r in results
                   for n in [3, 4, 5]
                   if f"{n}_options" in r['results_by_num_options'])

    analysis['overall_stats'] = {
        'total_experiments': all_total,
        'successful_experiments': all_successful,
        'success_rate': all_successful / all_total if all_total > 0 else 0,
        'total_unique_questions': len(set(r['question'] for r in results))
    }

    return analysis

# 确保可视化文件名有时间戳
if 'timestamp' not in locals():
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

# 执行分析（支持从 results_json_path 恢复）
if 'results' in locals() and results:
    ablation_analysis = analyze_ablation_results(results)

    print("\n📈 消融实验分析")
    print("=" * 70)

    # 总体统计
    print("\n总体统计:")
    overall = ablation_analysis['overall_stats']
    print(f"  • 总实验数: {overall['total_experiments']}")
    print(f"  • 成功实验: {overall['successful_experiments']}")
    print(f"  • 成功率: {100 * overall['success_rate']:.1f}%")
    print(f"  • 独特问题数: {overall['total_unique_questions']}")

    # 按选项数统计
    print("\n按选项数统计:")
    for num_opts in [3, 4, 5]:
        num_opts_key = f"{num_opts}_options"
        stats = ablation_analysis['by_num_options'][num_opts_key]
        print(f"\n  {num_opts} 个选项:")
        print(f"    • 问题数: {stats['total_questions']}")
        print(f"    • 成功: {stats['successful']}/{stats['total_questions']}")
        if stats['successful'] > 0:
            print(f"    • 平均相似度: {stats.get('mean_similarity_avg', 0):.3f} ± {stats.get('mean_similarity_std', 0):.3f}")
            print(f"    • 平均耗时: {stats['mean_elapsed_time']:.1f}s")
            print(f"    • 难度分布: Easy={stats['difficulty_distribution']['easy']}, Medium={stats['difficulty_distribution']['medium']}, Hard={stats['difficulty_distribution']['hard']}")

    # 保存分析结果
    analysis_output_path = Path(config.output_dir) / f"ablation_analysis_{timestamp}.json"
    with open(analysis_output_path, 'w', encoding='utf-8') as f:
        # Convert to JSON serializable format
        json_analysis = json.loads(json.dumps(ablation_analysis, default=str))
        json.dump(json_analysis, f, indent=2, ensure_ascii=False)
    print(f"\n✓ 分析结果保存: {analysis_output_path}")
else:
    print("⚠️ 未找到 results，请先运行生成步骤或设置 results_json_path。")

In [ ]:
# ============================================================================
# 部分 11: 可视化
# ============================================================================

# 可选：如果断开后 ablation_analysis 丢失，可从 results 重新计算，或从 JSON 重新加载
# 若需要从 JSON 加载，请设置下方路径
analysis_json_path = None  # 例如 Path(config.output_dir)/"ablation_analysis_20250101_120000.json"
results_json_path_for_plot = None  # 若 ablation_analysis 不在，且需要先加载 results 再计算

if 'ablation_analysis' not in locals():
    if analysis_json_path:
        with open(analysis_json_path, 'r', encoding='utf-8') as f:
            ablation_analysis = json.load(f)
        print(f"🔁 已从 JSON 加载 ablation_analysis: {analysis_json_path}")
    elif 'results' in locals() and results:
        ablation_analysis = analyze_ablation_results(results)
        print("🔁 已根据现有 results 重新计算 ablation_analysis")
    elif results_json_path_for_plot:
        with open(results_json_path_for_plot, 'r', encoding='utf-8') as f:
            results = json.load(f)
        ablation_analysis = analyze_ablation_results(results)
        print(f"🔁 已从 JSON 加载 results 并重新计算 ablation_analysis: {results_json_path_for_plot}")
    else:
        print("⚠️ 无 ablation_analysis，也未提供 JSON 路径。请先运行分析或提供路径。")

# 确保可视化文件名有时间戳
if 'timestamp' not in locals():
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')


def visualize_results(results: List[Dict], analysis: Dict, output_dir: str):
    """
    生成消融实验的可视化图表
    """
    
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    fig.suptitle('Counterfactual Options Generation - Ablation Study Results', fontsize=16, fontweight='bold')
    
    # 1. 成功率对比
    ax = axes[0, 0]
    num_options = [3, 4, 5]
    success_rates = []
    for num_opts in num_options:
        key = f"{num_opts}_options"
        stats = analysis['by_num_options'][key]
        rate = (stats['successful'] / stats['total_questions'] * 100) if stats['total_questions'] > 0 else 0
        success_rates.append(rate)
    
    ax.bar(num_options, success_rates, color=['#3498db', '#2ecc71', '#e74c3c'], alpha=0.7, edgecolor='black')
    ax.set_xlabel('Number of Options', fontsize=11, fontweight='bold')
    ax.set_ylabel('Success Rate (%)', fontsize=11, fontweight='bold')
    ax.set_title('Success Rate by Number of Options', fontsize=12, fontweight='bold')
    ax.set_ylim([0, 105])
    ax.grid(alpha=0.3, axis='y')
    for i, v in enumerate(success_rates):
        ax.text(num_options[i], v + 2, f'{v:.1f}%', ha='center', fontweight='bold')
    
    # 2. 平均相似度对比
    ax = axes[0, 1]
    mean_sims = []
    for num_opts in num_options:
        key = f"{num_opts}_options"
        stats = analysis['by_num_options'][key]
        mean_sims.append(stats.get('mean_similarity_avg', 0))
    
    ax.bar(num_options, mean_sims, color=['#3498db', '#2ecc71', '#e74c3c'], alpha=0.7, edgecolor='black')
    ax.set_xlabel('Number of Options', fontsize=11, fontweight='bold')
    ax.set_ylabel('Mean Similarity', fontsize=11, fontweight='bold')
    ax.set_title('Average Similarity Score by Number of Options', fontsize=12, fontweight='bold')
    ax.set_ylim([0, 1])
    ax.grid(alpha=0.3, axis='y')
    ax.axhline(y=config.min_similarity, color='r', linestyle='--', label=f'Min threshold ({config.min_similarity})', alpha=0.5)
    ax.axhline(y=config.max_similarity, color='b', linestyle='--', label=f'Max threshold ({config.max_similarity})', alpha=0.5)
    ax.legend()
    for i, v in enumerate(mean_sims):
        ax.text(num_options[i], v + 0.02, f'{v:.3f}', ha='center', fontweight='bold')
    
    # 3. 难度分布
    ax = axes[1, 0]
    difficulty_labels = ['Easy\n(>0.6)', 'Medium\n(0.4-0.6)', 'Hard\n(<0.4)']
    x = np.arange(len(num_options))
    width = 0.25
    
    easy_counts = []
    medium_counts = []
    hard_counts = []
    
    for num_opts in num_options:
        key = f"{num_opts}_options"
        diff = analysis['by_num_options'][key]['difficulty_distribution']
        easy_counts.append(diff['easy'])
        medium_counts.append(diff['medium'])
        hard_counts.append(diff['hard'])
    
    ax.bar(x - width, easy_counts, width, label='Easy', color='#2ecc71', alpha=0.8)
    ax.bar(x, medium_counts, width, label='Medium', color='#f39c12', alpha=0.8)
    ax.bar(x + width, hard_counts, width, label='Hard', color='#e74c3c', alpha=0.8)
    
    ax.set_xlabel('Number of Options', fontsize=11, fontweight='bold')
    ax.set_ylabel('Count', fontsize=11, fontweight='bold')
    ax.set_title('Difficulty Distribution', fontsize=12, fontweight='bold')
    ax.set_xticks(x)
    ax.set_xticklabels(num_options)
    ax.legend()
    ax.grid(alpha=0.3, axis='y')
    
    # 4. 耗时对比
    ax = axes[1, 1]
    elapsed_times = []
    for num_opts in num_options:
        key = f"{num_opts}_options"
        stats = analysis['by_num_options'][key]
        elapsed_times.append(stats['mean_elapsed_time'])
    
    ax.bar(num_options, elapsed_times, color=['#3498db', '#2ecc71', '#e74c3c'], alpha=0.7, edgecolor='black')
    ax.set_xlabel('Number of Options', fontsize=11, fontweight='bold')
    ax.set_ylabel('Elapsed Time (seconds)', fontsize=11, fontweight='bold')
    ax.set_title('Average Processing Time by Number of Options', fontsize=12, fontweight='bold')
    ax.grid(alpha=0.3, axis='y')
    for i, v in enumerate(elapsed_times):
        ax.text(num_options[i], v + 1, f'{v:.1f}s', ha='center', fontweight='bold')
    
    plt.tight_layout()
    
    # 保存图表
    plot_path = Path(output_dir) / f"ablation_visualization_{timestamp}.png"
    plt.savefig(plot_path, dpi=150, bbox_inches='tight')
    print(f"✓ 可视化已保存: {plot_path}")
    
    plt.show()

# 执行可视化（支持从 JSON 恢复）
if 'ablation_analysis' in locals():
    visualize_results(results if 'results' in locals() else [], ablation_analysis, config.output_dir)
else:
    print("⚠️ 无 ablation_analysis，无法绘图，请先提供 JSON 或重新计算。")

In [ ]:
# ============================================================================
# 部分 12: 全数据集处理（可选）
# ============================================================================

def process_full_dataset(qa_data_list: List[Dict], num_options_list: List[int]):
    """
    处理完整数据集
    
    注意: 这会消耗大量 API 配额和时间
    """
    
    print("\n⚠️ 警告: 这将处理所有视频和问题")
    print(f"   预计 API 调用次数: {sum(len(v['questions']) for v in qa_data_list) * len(num_options_list)}")
    
    total_questions = sum(len(v['questions']) for v in qa_data_list)
    print(f"   总问题数: {total_questions}")
    
    # 执行处理
    all_results = process_video_batch(qa_data_list, num_options_list)
    
    # 保存结果
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    
    # JSON
    json_output_path = Path(config.output_dir) / f"full_generated_options_{timestamp}.json"
    with open(json_output_path, 'w', encoding='utf-8') as f:
        json.dump(all_results, f, indent=2, ensure_ascii=False)
    print(f"✓ 完整结果已保存: {json_output_path}")
    
    # CSV（改进版 - 每个选项单独一列）
    csv_data = []
    for result in all_results:
        for num_opts_key, num_opts_result in result['results_by_num_options'].items():
            if num_opts_result['status'] == 'success':
                num_options = int(num_opts_key.split('_')[0])
                generated_opts = num_opts_result['generated_options']
                
                # 构建行数据
                row_data = {
                    'video_id': result['video_id'],
                    'question_id': result['q_id'],
                    'question': result['question'],
                    'category': result['category'],
                    'correct_answer': result['correct_answer'],
                    'num_options': num_options,
                }
                
                # 为每个选项创建单独的列（支持最多4个错误选项）
                for i in range(4):
                    if i < len(generated_opts):
                        row_data[f'option_{i+1}'] = generated_opts[i]
                        row_data[f'option_{i+1}_similarity'] = num_opts_result['similarities'].get(generated_opts[i], 0.0)
                    else:
                        row_data[f'option_{i+1}'] = None
                        row_data[f'option_{i+1}_similarity'] = None
                
                # 添加统计信息
                row_data.update({
                    'min_similarity': num_opts_result['similarity_stats']['min'],
                    'max_similarity': num_opts_result['similarity_stats']['max'],
                    'mean_similarity': num_opts_result['similarity_stats']['mean'],
                    'std_similarity': num_opts_result['similarity_stats']['std'],
                    'elapsed_time': num_opts_result['elapsed_time']
                })
                
                csv_data.append(row_data)
    
    if csv_data:
        csv_output_path = Path(config.output_dir) / f"full_generated_options_{timestamp}.csv"
        csv_df = pd.DataFrame(csv_data)
        csv_df.to_csv(csv_output_path, index=False, encoding='utf-8', quoting=1, line_terminator='\n')
        print(f"✓ CSV 已保存: {csv_output_path}")
        print(f"  每个选项单独一列，包含相似度分数")
        print(f"\n📋 CSV 列结构:")
        print(f"   基础列: video_id, question_id, question, category, correct_answer, num_options")
        print(f"   选项列: option_1, option_1_similarity, option_2, option_2_similarity, ...")
        print(f"   统计列: min_similarity, max_similarity, mean_similarity, std_similarity, elapsed_time")
    
    # 分析
    full_analysis = analyze_ablation_results(all_results)
    analysis_output_path = Path(config.output_dir) / f"full_ablation_analysis_{timestamp}.json"
    with open(analysis_output_path, 'w', encoding='utf-8') as f:
        json_analysis = json.loads(json.dumps(full_analysis, default=str))
        json.dump(json_analysis, f, indent=2, ensure_ascii=False)
    print(f"✓ 分析已保存: {analysis_output_path}")
    
    # 可视化
    visualize_results(all_results, full_analysis, config.output_dir)
    
    return all_results, full_analysis

# 函数已定义，可以用以下方式调用:
# full_results, full_analysis = process_full_dataset(qa_pairs, config.num_options_list)

print("✅ 全数据集处理函数已定义")

# 快速参考指南

## 使用方法

### 基本流程

1. **配置** (部分 2)
   - API Key 会自动从 Colab Secrets 加载
   - 可修改 `config` 对象来调整参数

2. **加载数据** (部分 3-4)
   - 自动加载 `QA_pair.csv` 和视频文件
   - 验证数据完整性

3. **生成选项** (部分 9)
   - 调用 Qwen VL API
   - 为每个问题生成 3, 4, 5 个选项
   - 自动计算相似度

4. **分析结果** (部分 10-11)
   - 消融实验分析
   - 可视化对比图表

### 主要参数

```python
config.num_options_list = [3, 4, 5]      # 选项数（用于消融实验）
config.min_similarity = 0.2               # 最小相似度阈值
config.max_similarity = 0.8               # 最大相似度阈值
config.max_frames_per_video = 8           # 每个视频最多提取的帧数
config.target_fps = 2                     # 帧采样目标速率
```

### 调整提示词

编辑部分 5 的 `create_option_generation_prompt()` 函数来自定义选项生成策略。

## 输出文件

- `generated_options_YYYYMMDD_HHMMSS.json` - 详细的生成结果
- `generated_options_YYYYMMDD_HHMMSS.csv` - 结构化的选项数据
- `ablation_analysis_YYYYMMDD_HHMMSS.json` - 消融实验分析
- `ablation_visualization_YYYYMMDD_HHMMSS.png` - 可视化图表

## 注意事项

- 首次运行会下载相关模型和资源
- API 调用有配额限制，请注意成本
- 建议先用小数据集测试，再运行完整数据集
- 相似度计算基于 TF-IDF 向量化，可根据需要修改